# Anomaly Detection

In [54]:
import  pandas as pd
import numpy as np
import scipy.stats as ss

In [55]:
pd.options.mode.chained_assignment = None

# Importing data

1. We have imported **56133 records** with **35 columns**.
2. Data ranges from **April 3, 2018 to April 8, 2020**.
3. We have **463 unique distributors** in **Southern region** of India in **General Trade**

In [56]:
data = pd.read_excel(r"D:\Analytics\Undercutting\South region - GT\Data\Undercutting South region GT 2018-2020.xlsx",sheet_name= "Sheet1")

In [57]:
data.shape

(56133, 35)

In [58]:
data.head(2)

,Distribution Channel,Division,Old Sold-To Party,Sold-To Party,Customer Group 3,Price List Type,Region Order,Plant,Material,Date,...,Gross,Gross.1,Sales cost,Sales cost.1,Net,Net.1,RtnsQty,RtnsQty.1,Rtns (cst),Rtns (cst).1
0,GT General Trade,PC Personal Care,2000001 N.CT. AGENCIES,2000001 N.CT. AGENCIES,006 GT,02 Super Stockist,TN TN,PYC1 CAVINKARE PVT LTD,KH0180HWP02R KARTHIKA HERBAL POWDER CONT...,23.10.2019,...,187.0,KG,15169.7,INR,144.0,KG,0.0,CV,0.0,INR
1,GT General Trade,PC Personal Care,2000001 N.CT. AGENCIES,2000001 N.CT. AGENCIES,006 GT,02 Super Stockist,TN TN,PYC1 CAVINKARE PVT LTD,KH05D8HWP02R KARTHIKA HERBAL POWDER 5.8 ...,10.07.2019,...,5040.0,KG,272536.0,INR,4640.0,KG,0.0,CV,0.0,INR


In [59]:
data.columns

Index(['Distribution Channel', 'Division', 'Old Sold-To Party',
       'Sold-To Party', 'Customer Group 3', 'Price List Type', 'Region Order',
       'Plant', 'Material', 'Date', 'Sales', 'Sales.1', 'Returns',
       'Cred.Memos', 'Cred.Memos.1', 'CM: net 1', 'CM: net 1.1', 'Net Value',
       'Net Value.1', 'Tax', 'Tax.1', 'Bill. Qty', 'Bill. Qty.1', 'CredMemQty',
       'CredMemQty.1', 'Gross', 'Gross.1', 'Sales cost', 'Sales cost.1', 'Net',
       'Net.1', 'RtnsQty', 'RtnsQty.1', 'Rtns (cst)', 'Rtns (cst).1'],
      dtype='object')

# Data Transformation and Feature Engineering

In [60]:
temp                   =  data["Material"].str.split("       ",expand=True)
data["Prod_Code"]      = temp[0]
data["Prod_Name"]      = temp[1]
data["Date"]           = pd.to_datetime(data["Date"],format='%d.%m.%Y')
# data["Month"]          = data["New date"].dt.strftime('%B')

In [61]:
dist_data = data[["Sold-To Party","Date","Prod_Name","Bill. Qty"]]

In [62]:
dist_data = dist_data[dist_data["Bill. Qty"] != 0]

In [63]:
dist_data.shape

(54133, 4)

In [64]:
dist_data.head(2)

,Sold-To Party,Date,Prod_Name,Bill. Qty
0,2000001 N.CT. AGENCIES,2019-10-23,KARTHIKA HERBAL POWDER CONT 180G 80PCS,10.0
1,2000001 N.CT. AGENCIES,2019-07-10,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,400.0


## Removing single records

In [65]:
rmv_single_rows = dist_data.groupby(["Sold-To Party","Prod_Name"]).agg(Count = ("Prod_Name","count")).reset_index()
rmv_single_rows.head(2)

,Sold-To Party,Prod_Name,Count
0,2000001 N.CT. AGENCIES,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,104
1,2000001 N.CT. AGENCIES,KARTHIKA HERBAL POWDER 50GM 180 PCS,24


In [66]:
len(rmv_single_rows[rmv_single_rows["Count"] == 1])

294

In [67]:
dist_data_clean = pd.merge(dist_data,rmv_single_rows,on = ["Sold-To Party","Prod_Name"],how = "left")
dist_data_clean = dist_data_clean[dist_data_clean["Count"] != 1]
dist_data_clean.shape

(53839, 5)

In [68]:
print("So totally {a} rows will be dropped from total of {b} rows. Because it has only 1 data".format(a = len(rmv_single_rows[rmv_single_rows["Count"] == 1]), b= len(dist_data)))

So totally 294 rows will be dropped from total of 54133 rows. Because it has only 1 data


In [69]:
dist_data_clean.head(2)

,Sold-To Party,Date,Prod_Name,Bill. Qty,Count
0,2000001 N.CT. AGENCIES,2019-10-23,KARTHIKA HERBAL POWDER CONT 180G 80PCS,10.0,60
1,2000001 N.CT. AGENCIES,2019-07-10,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,400.0,104


After removing we have 56133 records for analysis

# Descriptive Analytics

In [70]:
dist_data_descriptive = dist_data_clean.groupby(["Sold-To Party","Prod_Name"]).agg(Mean = ("Bill. Qty","mean"), 
                                           stddev = ("Bill. Qty","std"),Median =("Bill. Qty","median")).reset_index()
dist_data_descriptive.isna().sum()

Sold-To Party    0
Prod_Name        0
Mean             0
stddev           0
Median           0
dtype: int64

In [71]:
zscore_data = pd.merge(dist_data_clean, dist_data_descriptive, on = ["Sold-To Party","Prod_Name"], how = 'left')

In [72]:
# Test Data
zscore_data[(zscore_data["Sold-To Party"] == "2000001    N.CT. AGENCIES") & (zscore_data["Prod_Name"] == "KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS")].head(2)

,Sold-To Party,Date,Prod_Name,Bill. Qty,Count,Mean,stddev,Median
1,2000001 N.CT. AGENCIES,2019-07-10,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,400.0,104,178.009615,156.68957,146.5
2,2000001 N.CT. AGENCIES,2019-07-16,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,150.0,104,178.009615,156.68957,146.5


In [73]:
zscore_data.isna().sum()
# zscore_data.dropna(axis = 0, inplace = True)

Sold-To Party    0
Date             0
Prod_Name        0
Bill. Qty        0
Count            0
Mean             0
stddev           0
Median           0
dtype: int64

# Outlier Detection

### Method 1 - Z Score Calculation

Z score = (Observation - Mean)/Sigma

In [74]:
zscore_data['z-score']= (zscore_data['Bill. Qty'] - zscore_data['Mean'])/zscore_data['stddev']

In [75]:
# Test Data
zscore_data[(zscore_data["Sold-To Party"] == "2000001    N.CT. AGENCIES") & (zscore_data["Prod_Name"] == "KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS")].tail(2)

,Sold-To Party,Date,Prod_Name,Bill. Qty,Count,Mean,stddev,Median,z-score
24081,2000001 N.CT. AGENCIES,2019-03-19,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,80.0,104,178.009615,156.68957,146.5,-0.625502
24082,2000001 N.CT. AGENCIES,2019-03-27,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,100.0,104,178.009615,156.68957,146.5,-0.497861


In [76]:
zscore_data.sort_values(by='z-score', ascending=False,inplace=True)
zscore_data.head(5)

,Sold-To Party,Date,Prod_Name,Bill. Qty,Count,Mean,stddev,Median,z-score
9,2000001 N.CT. AGENCIES,2019-09-18,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,1300.0,104,178.009615,156.689570,146.5,7.160594
10078,2000131 SRI PONNIAMMAN AGENCY,2019-04-10,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,113.0,92,17.989130,14.311676,15.0,6.638696
29571,2000080 DHARSHU AGENCY,2018-12-03,MEERA HERBAL POWDER 120GM CONT 60 PCS,52.0,49,3.734694,7.347833,2.0,6.568645
32390,2000096 VICTORY ENTERPRISE,2018-09-18,MEERA HERBAL POWDER 120GM CONT 60 PCS,50.0,56,7.339286,6.506715,5.5,6.556414
9915,2000130 S.P. AGENCIES,2019-10-16,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,250.0,90,41.577778,32.032279,32.5,6.506631


### Method 2 - Mod Z Score Calculation - IBM formula

MAD - Median Absolute deviation        
MAD = Median(ABS(observation - Median))       

MeanAD - Mean Absolute deviation     
MeanAD = Mean(ABS(Observation - Mean))      
            

In [77]:
zscore_data['MAD Bill. Qty'] = np.absolute(zscore_data['Bill. Qty'] - zscore_data['Median']) #abs(x-median)

zscore_data['MeanAD Bill. Qty'] = np.absolute(zscore_data['Bill. Qty'] - zscore_data['Mean']) #abs(x-mean)

In [78]:
zscore_data.head(2)

,Sold-To Party,Date,Prod_Name,Bill. Qty,Count,Mean,stddev,Median,z-score,MAD Bill. Qty,MeanAD Bill. Qty
9,2000001 N.CT. AGENCIES,2019-09-18,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,1300.0,104,178.009615,156.689570,146.5,7.160594,1153.5,1121.990385
10078,2000131 SRI PONNIAMMAN AGENCY,2019-04-10,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,113.0,92,17.989130,14.311676,15.0,6.638696,98.0,95.010870


In [79]:
MeanAD_MAD_Group = zscore_data.groupby(["Sold-To Party","Prod_Name"]).agg(MAD = ("MAD Bill. Qty","median"),
                                                                          MeanAD = ("MeanAD Bill. Qty","mean")).reset_index()

In [80]:
MeanAD_MAD_Group.head(2)

,Sold-To Party,Prod_Name,MAD,MeanAD
0,2000001 N.CT. AGENCIES,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,72.5,101.819527
1,2000001 N.CT. AGENCIES,KARTHIKA HERBAL POWDER 50GM 180 PCS,0.5,3.763889


In [81]:
zscore_data = pd.merge(zscore_data,MeanAD_MAD_Group,on=["Sold-To Party","Prod_Name"],how='left')

In [82]:
# zscore_data.to_excel(r"D:\Analytics\Undercutting\South region - GT\Outfiles\zscoredata1.xlsx",index = False)

# Mod Z Score formula:

**Formula1** : If MAD = 0 then, (X-MED)/(1.253314 * MeanAD)   
**Formula2** : If MAD != 0 then, (X-MED)/(1.486 * MAD)   


In [83]:
MAD_data_equal0   = zscore_data[zscore_data['MAD'] == 0]
MAD_data_greater0 = zscore_data[zscore_data['MAD']  > 0]

In [84]:
MAD_data_equal0.shape

(14480, 13)

In [85]:
MAD_data_greater0.shape

(39359, 13)

So we will using Formula 1 for 27% of the records and formula 2 for the rest 73% of the records

**Formula1** : If Median = 0 then, (X-MED)/(1.253314 * MeanAD)  

In [86]:
MAD_data_equal0['Mod-z_score'] = MAD_data_equal0['MAD Bill. Qty']/(1.253314 * MAD_data_equal0['MeanAD']) 

In [87]:
MAD_data_equal0[(MAD_data_equal0["Sold-To Party"] == "2000727    SRI VEERABHADRA AGENCIES") &
                 (MAD_data_equal0["Prod_Name"] == "MEERA ADVANCE HW POWDER 120GM CONT 60 PC")]

,Sold-To Party,Date,Prod_Name,Bill. Qty,Count,Mean,stddev,Median,z-score,MAD Bill. Qty,MeanAD Bill. Qty,MAD,MeanAD,Mod-z_score
3465,2000727 SRI VEERABHADRA AGENCIES,2019-07-30,MEERA ADVANCE HW POWDER 120GM CONT 60 PC,4.0,9,2.333333,1.0,2.0,1.666667,2.0,1.666667,0.0,0.740741,2.154289
3466,2000727 SRI VEERABHADRA AGENCIES,2019-12-25,MEERA ADVANCE HW POWDER 120GM CONT 60 PC,4.0,9,2.333333,1.0,2.0,1.666667,2.0,1.666667,0.0,0.740741,2.154289
27850,2000727 SRI VEERABHADRA AGENCIES,2020-03-17,MEERA ADVANCE HW POWDER 120GM CONT 60 PC,2.0,9,2.333333,1.0,2.0,-0.333333,0.0,0.333333,0.0,0.740741,0.000000
27851,2000727 SRI VEERABHADRA AGENCIES,2019-07-16,MEERA ADVANCE HW POWDER 120GM CONT 60 PC,2.0,9,2.333333,1.0,2.0,-0.333333,0.0,0.333333,0.0,0.740741,0.000000
27852,2000727 SRI VEERABHADRA AGENCIES,2019-12-18,MEERA ADVANCE HW POWDER 120GM CONT 60 PC,2.0,9,2.333333,1.0,2.0,-0.333333,0.0,0.333333,0.0,0.740741,0.000000
27853,2000727 SRI VEERABHADRA AGENCIES,2019-11-20,MEERA ADVANCE HW POWDER 120GM CONT 60 PC,2.0,9,2.333333,1.0,2.0,-0.333333,0.0,0.333333,0.0,0.740741,0.000000
27854,2000727 SRI VEERABHADRA AGENCIES,2019-11-27,MEERA ADVANCE HW POWDER 120GM CONT 60 PC,2.0,9,2.333333,1.0,2.0,-0.333333,0.0,0.333333,0.0,0.740741,0.000000
27855,2000727 SRI VEERABHADRA AGENCIES,2019-09-04,MEERA ADVANCE HW POWDER 120GM CONT 60 PC,2.0,9,2.333333,1.0,2.0,-0.333333,0.0,0.333333,0.0,0.740741,0.000000
50363,2000727 SRI VEERABHADRA AGENCIES,2019-07-23,MEERA ADVANCE HW POWDER 120GM CONT 60 PC,1.0,9,2.333333,1.0,2.0,-1.333333,1.0,1.333333,0.0,0.740741,1.077144


In [88]:
# MAD_data_equal0.to_excel(r"D:\Analytics\Undercutting\South region - GT\Outfiles\MAD_Data_equal_0.xlsx",index = False)

In [89]:
MAD_data_greater0['Mod-z_score'] = MAD_data_greater0['MAD Bill. Qty']/(1.486 * MAD_data_greater0['MAD']) 

In [90]:
MAD_data_greater0.head(5)

,Sold-To Party,Date,Prod_Name,Bill. Qty,Count,Mean,stddev,Median,z-score,MAD Bill. Qty,MeanAD Bill. Qty,MAD,MeanAD,Mod-z_score
0,2000001 N.CT. AGENCIES,2019-09-18,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,1300.0,104,178.009615,156.689570,146.5,7.160594,1153.5,1121.990385,72.5,101.819527,10.706827
1,2000131 SRI PONNIAMMAN AGENCY,2019-04-10,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,113.0,92,17.989130,14.311676,15.0,6.638696,98.0,95.010870,5.0,9.073960,13.189771
2,2000080 DHARSHU AGENCY,2018-12-03,MEERA HERBAL POWDER 120GM CONT 60 PCS,52.0,49,3.734694,7.347833,2.0,6.568645,50.0,48.265306,1.0,2.946272,33.647376
3,2000096 VICTORY ENTERPRISE,2018-09-18,MEERA HERBAL POWDER 120GM CONT 60 PCS,50.0,56,7.339286,6.506715,5.5,6.556414,44.5,42.660714,2.0,3.316964,14.973082
4,2000130 S.P. AGENCIES,2019-10-16,KARTHIKA HERBAL POWDER 5.8 GM 2000 PCS,250.0,90,41.577778,32.032279,32.5,6.506631,217.5,208.422222,14.0,21.471111,10.454720


In [91]:
# MAD_data_greater0.to_excel(r"D:\Analytics\Undercutting\South region - GT\Outfiles\MAD_data_greater0.xlsx",index = False)

In [92]:
zscore_data = MAD_data_equal0.append(MAD_data_greater0)

In [93]:
zscore_data.isna().sum()

Sold-To Party          0
Date                   0
Prod_Name              0
Bill. Qty              0
Count                  0
Mean                   0
stddev                 0
Median                 0
z-score             2199
MAD Bill. Qty          0
MeanAD Bill. Qty       0
MAD                    0
MeanAD                 0
Mod-z_score         2199
dtype: int64

In [94]:
zscore_data["z-score"].fillna(0,inplace=True)

In [95]:
zscore_data["Mod-z_score"].fillna(0,inplace= True)

In [96]:
# zscore_data.to_excel(r"D:\Analytics\Undercutting\South region - GT\Data\zscore_data.xlsx",index = False)

# Distributor Level data

In [97]:
distributors_unique         = pd.DataFrame(data["Sold-To Party"].unique())
distributors_unique         = distributors_unique.rename(columns = {0 : "Sold-To Party"})
new                         = distributors_unique["Sold-To Party"].str.split("    ",expand = True)
dist_ID                     = pd.DataFrame()
dist_ID["Account"]          = new[0]
dist_ID["Distributor name"] = new[1]
Master_distributor_data     = pd.concat([distributors_unique,dist_ID],axis=1)

In [98]:
# Master_distributor_data.to_excel(r"D:\Analytics\Undercutting\South region - GT\Data\Distributors data.xlsx",index = False)

In [99]:
materials_unique = pd.DataFrame(data["Material"].unique())

In [100]:
# materials_unique.to_excel(r"D:\Analytics\Undercutting\South region - GT\Data\Materials.xlsx",index = False)

## MADe Calculation

**FORMULA**     
1. **2MADe Method = Median +/- 2MADe**     
2. **3MADe Method = Median +/- 3MADe**            
where MADe = 1.483 × MAD     
      MAD = median {|xi −median(x)|}, i = 1,2,...,n

In [113]:
zscore_data['MADe']     = 1.483 * zscore_data['MAD']
zscore_data['Lower_lt'] = zscore_data['Median'] - (3 * zscore_data['MADe'])
zscore_data['Upper_lt'] = zscore_data['Median'] + (3 * zscore_data['MADe'])
zscore_data['3MADe']    = np.where((zscore_data['Bill. Qty'] < zscore_data['Lower_lt']) |
                                    (zscore_data['Bill. Qty'] > zscore_data['Upper_lt']),
                                    "Outlier",
                                    "Not Outlier")

In [114]:
zscore_data.head()

,Sold-To Party,Date,Prod_Name,Bill. Qty,Count,Mean,stddev,Median,z-score,MAD Bill. Qty,MeanAD Bill. Qty,MAD,MeanAD,Mod-z_score,MADe,Lower_lt,Upper_lt,3MADe
8,2000098 ZARA DISTRIBUTORS,2019-04-27,KARTHIKA HERBAL POWDER 50GM 180 PCS,16.0,41,1.731707,2.356103,1.0,6.055888,15.0,14.268293,0.0,0.963712,12.418927,0.0,1.0,1.0,Outlier
9,2000090 SRI SRINIVASA AGENCIES,2019-09-27,KARTHIKA HERBAL POWDER CONT 180G 80PCS,8.0,53,1.471698,1.084928,1.0,6.017269,7.0,6.528302,0.0,0.658597,8.480436,0.0,1.0,1.0,Outlier
22,2000096 VICTORY ENTERPRISE,2019-03-08,KARTHIKA HERBAL POWDER CONT 180G 80PCS,6.0,49,1.428571,0.841625,1.0,5.431667,5.0,4.571429,0.0,0.577259,6.910971,0.0,1.0,1.0,Outlier
32,2000094 T.L AGENCIES,2019-07-13,KARTHIKA HERBAL POWDER 50GM 180 PCS,3.0,29,1.068966,0.371391,1.0,5.199469,2.0,1.931034,0.0,0.133175,11.982518,0.0,1.0,1.0,Outlier
37,2000097 VMR TRADERS,2019-03-27,KARTHIKA HERBAL POWDER 50GM 180 PCS,3.0,47,1.106383,0.374981,1.0,5.049905,2.0,1.893617,0.0,0.194658,8.197801,0.0,1.0,1.0,Outlier


In [115]:
zscore_data.to_excel(r"D:\Analytics\Undercutting\South region - GT\Data\zscore_data.xlsx",index = False)